In [4]:
pip install --upgrade scikit-learn

In [2]:
import os
import pandas as pd
import numpy as np
import mlflow
import mlflow.sklearn
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import OrdinalEncoder
from sklearn.impute import SimpleImputer

In [3]:
# Charger les données
chemin_dossier = "C:/Users/paulm/Documents/GitHub/Projet7/data/"
df = pd.read_csv(os.path.join(chemin_dossier, 'processed_data.csv'))

# Sélectionner les colonnes catégorielles
colonnes_categorielles = df.select_dtypes(include='object').columns.tolist()

# Afficher le nombre de valeurs uniques dans chaque colonne catégorielle
for col in colonnes_categorielles:
    nb_valeurs_uniques = df[col].nunique()
    print(f"Colonne '{col}' contient {nb_valeurs_uniques} valeurs uniques.")


Colonne 'NAME_CONTRACT_TYPE' contient 2 valeurs uniques.
Colonne 'CODE_GENDER' contient 3 valeurs uniques.
Colonne 'FLAG_OWN_CAR' contient 2 valeurs uniques.
Colonne 'FLAG_OWN_REALTY' contient 2 valeurs uniques.
Colonne 'NAME_TYPE_SUITE' contient 7 valeurs uniques.
Colonne 'NAME_INCOME_TYPE' contient 8 valeurs uniques.
Colonne 'NAME_EDUCATION_TYPE' contient 5 valeurs uniques.
Colonne 'NAME_FAMILY_STATUS' contient 6 valeurs uniques.
Colonne 'NAME_HOUSING_TYPE' contient 6 valeurs uniques.
Colonne 'OCCUPATION_TYPE' contient 18 valeurs uniques.
Colonne 'WEEKDAY_APPR_PROCESS_START' contient 7 valeurs uniques.
Colonne 'ORGANIZATION_TYPE' contient 58 valeurs uniques.
Colonne 'FONDKAPREMONT_MODE' contient 4 valeurs uniques.
Colonne 'HOUSETYPE_MODE' contient 3 valeurs uniques.
Colonne 'WALLSMATERIAL_MODE' contient 7 valeurs uniques.
Colonne 'EMERGENCYSTATE_MODE' contient 2 valeurs uniques.


In [ ]:
# Supprimer les colonnes avec trop de valeurs uniques
colonnes_a_supprimer = ['ORGANIZATION_TYPE', 'OCCUPATION_TYPE']
df.drop(columns=colonnes_a_supprimer, inplace=True)

In [8]:
# Identifier les colonnes contenant des valeurs infinies
infinite_cols = df.columns.to_series()[np.isinf(df).any()]

# Afficher les colonnes avec des valeurs infinies
print("Colonnes avec des valeurs infinies :")
print(infinite_cols)

# Identifier les lignes contenant des valeurs infinies
rows_with_infs = df.index[np.isinf(df).any(axis=1)]

# Afficher les indices des lignes contenant des valeurs infinies
print("\nIndices des lignes avec des valeurs infinies :")
print(rows_with_infs.tolist())

# Supprimer les lignes contenant des valeurs infinies
df.drop(rows_with_infs, inplace=True)

# Vérifier si des valeurs infinies existent encore après suppression
print("\nReste-t-il des valeurs infinies ?")
print(np.isinf(df).any().any())

Colonnes avec des valeurs infinies :
PREV_APP_CREDIT_PERC_MAX      PREV_APP_CREDIT_PERC_MAX
PREV_APP_CREDIT_PERC_MEAN    PREV_APP_CREDIT_PERC_MEAN
INSTAL_PAYMENT_PERC_MEAN      INSTAL_PAYMENT_PERC_MEAN
INSTAL_PAYMENT_PERC_SUM        INSTAL_PAYMENT_PERC_SUM
dtype: object

Indices des lignes avec des valeurs infinies :
[5687, 60477, 79077, 89018, 98509, 126768, 128791, 140426, 152087, 167136, 199103, 201086, 236164, 238381, 272829, 277962, 287300, 292852, 305373]

Reste-t-il des valeurs infinies ?
False


In [10]:
cost_fp = 1  # Coût d'un faux positif
cost_fn = 10  # Coût d'un faux négatif (10 fois le coût d'un FP)

# Vérifier les types de données et les valeurs
# Assurez-vous que les colonnes non numériques sont correctement gérées avant l'encodage
non_numeric_cols = df.select_dtypes(exclude=[np.number]).columns
df.drop(columns=non_numeric_cols, inplace=True)

# Encodage des variables catégorielles avec OrdinalEncoder
ordinal_encoder = OrdinalEncoder()
df_encoded = pd.DataFrame(ordinal_encoder.fit_transform(df), columns=df.columns)

# Gestion des valeurs manquantes avec SimpleImputer
imputer = SimpleImputer(strategy='mean')
X = pd.DataFrame(imputer.fit_transform(df_encoded.drop(columns=['TARGET'])), columns=df_encoded.drop(columns=['TARGET']).columns)

# Séparer les caractéristiques et la cible
y = df_encoded['TARGET']

# Séparer les données en ensemble d'entraînement et de test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Définir les modèles à tester avec leurs paramètres
models = [
    ('Random Forest', RandomForestClassifier(class_weight='balanced', random_state=42))
]

params = {
    'Random Forest': {'n_estimators': [50, 100], 'max_depth': [None, 10, 20]}
}

# Configurer l'expérience MLflow
mlflow.set_experiment("Credit Scoring Experiment")

# Échantillonner les données
sample_size = 0.1  # Utiliser 10% des données pour un échantillonnage rapide
X_train_sample, _, y_train_sample, _ = train_test_split(X_train, y_train, test_size=(1 - sample_size), random_state=42)
X_test_sample, _, y_test_sample, _ = train_test_split(X_test, y_test, test_size=(1 - sample_size), random_state=42)

# Tester les modèles et trouver les meilleurs paramètres
for name, model in models:
    with mlflow.start_run(run_name=name):
        print(f"Entraînement du modèle {name}...")

        grid_search = GridSearchCV(estimator=model, param_grid=params[name], scoring='f1', cv=3)
        grid_search.fit(X_train_sample, y_train_sample)

        # Loguer les paramètres dans MLflow
        mlflow.log_params(grid_search.best_params_)

        # Afficher les meilleurs paramètres et le score F1
        print(f"Meilleurs paramètres pour {name}: {grid_search.best_params_}")
        print(f"Score F1 moyen sur le jeu de validation: {grid_search.best_score_:.3f}")

        # Utiliser le meilleur modèle pour les prédictions
        best_model = grid_search.best_estimator_
        y_pred = best_model.predict(X_test_sample)

        # Évaluer le modèle avec le meilleur seuil pour minimiser le coût métier
        y_prob = best_model.predict_proba(X_test_sample)[:, 1]
        thresholds = np.linspace(0, 1, 100)
        costs = []
        for threshold in thresholds:
            y_pred_thresholded = (y_prob > threshold).astype(int)
            fp = np.sum((y_pred_thresholded == 1) & (y_test_sample == 0)) * cost_fp
            fn = np.sum((y_pred_thresholded == 0) & (y_test_sample == 1)) * cost_fn
            total_cost = fp + fn
            costs.append(total_cost)

        best_threshold = thresholds[np.argmin(costs)]
        print(f"Meilleur seuil pour minimiser le coût métier avec {name}: {best_threshold}")

        y_pred_best_threshold = (y_prob > best_threshold).astype(int)

        # Afficher le rapport de classification et la matrice de confusion
        classification_report_str = classification_report(y_test_sample, y_pred_best_threshold)
        confusion_matrix_str = confusion_matrix(y_test_sample, y_pred_best_threshold)

        print(f"Rapport de classification avec le meilleur seuil pour {name}:")
        print(classification_report_str)

        print(f"Matrice de confusion avec le meilleur seuil pour {name}:")
        print(confusion_matrix_str)
        print()

        # Loguer les métriques et les résultats dans MLflow
        mlflow.log_metric("best_score_f1", grid_search.best_score_)
        mlflow.log_metric("best_threshold", best_threshold)
        mlflow.log_text(classification_report_str, "classification_report.txt")
        mlflow.log_text(str(confusion_matrix_str), "confusion_matrix.txt")

        # Loguer le modèle
        mlflow.sklearn.log_model(best_model, "model")


Entraînement du modèle Random Forest...
Meilleurs paramètres pour Random Forest: {'max_depth': 10, 'n_estimators': 100}
Score F1 moyen sur le jeu de validation: 0.246
Meilleur seuil pour minimiser le coût métier avec Random Forest: 0.38383838383838387
Rapport de classification avec le meilleur seuil pour Random Forest:
              precision    recall  f1-score   support

         0.0       0.96      0.72      0.82      5663
         1.0       0.16      0.62      0.25       486

    accuracy                           0.71      6149
   macro avg       0.56      0.67      0.54      6149
weighted avg       0.89      0.71      0.78      6149

Matrice de confusion avec le meilleur seuil pour Random Forest:
[[4082 1581]
 [ 187  299]]

